In [52]:
import pandas as pd
import numpy as np
import operator
import os
from pathlib import Path
cwd = os.getcwd() # get path of current working directory
main_folder = str(Path(cwd).parents[0]) # main folder is the lowest level in project

In [53]:
cex84_02 = pd.read_stata(main_folder + "\data\cex84_02.dta")
cex84_02 = cex84_02[cex84_02.year >= 94]
cex84_02 = cex84_02[cex84_02.year <= 97]
cex84_02['ee'] = 0
cex84_02.loc[(cex84_02['year'] <=95) & (cex84_02['educatio'] >=21), 'ee'] = 1
cex84_02.loc[(cex84_02['year'] >=96) & (cex84_02['educatio'] >=40), 'ee'] = 1
cex84_02['meanee'] = cex84_02['ee'].mean()
#cex84_02
cex84_02[['meanee']].describe()

,meanee
count,2.330100e+04
mean,5.216943e-01
std,1.036971e-13
min,5.216943e-01
25%,5.216943e-01
50%,5.216943e-01
75%,5.216943e-01
max,5.216943e-01


In [54]:
#/*expenditure used in this paper*/
cex84_02 = cex84_02.drop(columns=['autos'])
cex84_02['tt'] = cex84_02.loc[:, 'foodhome':'charity'].sum(axis=1)
cex84_02 = cex84_02[cex84_02.tt >= 0]
cex84_02['ltotexp'] = np.log(cex84_02['tt']) - np.log(cex84_02['cpi_97'])
#cex84_02

In [55]:
#MemoryError: Unable to allocate 4.04 GiB for an array with shape (23300, 23300) and data type float64
#cex84_02.loc[:, 'foodhome':'charity']/cex84_02['tt']
cex84_02_div = pd.read_stata('cex84_02_div.dta')
cex84_02_div = cex84_02_div.drop(columns=['gambling', 'toiletry', 'housuppl'])
#cex84_02_div

In [56]:
#Aggregate some small expenses with similar expenses
cex84_02_div['foodout'] = cex84_02_div['foodout'] + cex84_02_div['foodwork'] 
cex84_02_div['renthome'] = cex84_02_div['renthome'] + cex84_02_div['rentothr']
cex84_02_div['pubs'] = cex84_02_div['pubs'] + cex84_02_div['books']
cex84_02_div['hospital'] = cex84_02_div['hospital'] + cex84_02_div['nurshome']
cex84_02_div['masstran'] = cex84_02_div['masstran'] + cex84_02_div['tolls']
cex84_02_div = cex84_02_div.drop(columns=['foodwork', 'rentothr', 'books', 'nurshome', 'tolls'])

cex84_02_div['sex'] = cex84_02_div['sex'] - 1
#cex84_02_div

In [57]:
cex84_02_div.to_stata('sampleUS6')

In [58]:
#*shares
cex84_02_div.rename(columns={'foodhome':'share1', 'foodout':'share2', 'tobacco':'share3', 'alcohol':'share4', 'niteclub':'share5', 'clothes':'share6','tailors':'share7', 'jewelry':'share8', 'hlthbeau':'share9', 'renthome':'share10', 'furnish':'share11', 'elect':'share12', 'gas':'share13','water':'share14', 'homefuel':'share15','telephon':'share16', 'servants':'share17', 'drugs':'share18', 'orthopd':'share19', 'doctors':'share20', 'hospital':'share21', 'helthins':'share22', 'busiserv':'share23', 'lifeins':'share24', 'parts':'share25','carservs':'share26', 'gasoline':'share27', 'autoins':'share28', 'masstran':'share29', 'othtrans':'share30', 'airfare':'share31', 'pubs':'share32', 'recsport':'share33', 'othrec':'share34', 'highedu':'share35','lowedu':'share36', 'othedu':'share37', 'charity':'share38'}, inplace=True)

In [59]:
list_shares = ['share1', 'share2', 'share3', 'share4', 'share5', 'share6', 'share7', 'share8', 'share9','share10', 'share11',
               'share12', 'share13', 'share14', 'share15', 'share16', 'share17', 'share18', 'share19', 'share20', 'share21',
               'share22', 'share23', 'share24', 'share25', 'share26', 'share27', 'share28', 'share29', 'share30', 'share31', 
               'share32', 'share33', 'share34', 'share35', 'share36', 'share37', 'share38']

In [60]:
cex84_02_divg = cex84_02_div.groupby('year')[list_shares].mean()
cex84_02_divg = cex84_02_divg.reset_index()
cex84_02_divg['year'] = cex84_02_divg['year'] + 1900 
#cex84_02_divg

In [61]:
cex84_02_divg.to_stata('temp26')

In [62]:
#Construction of aggregate prices as weighted average of prices
pricesUS = pd.read_stata(main_folder + "\data\pricesUS.dta")
pricesUS = pricesUS.drop(0).replace('',0)
pricesUS = pricesUS.applymap(float)
#pricesUS

#for var c1-c126: replace X=X/X[140]
pricesUS_1 = pricesUS.drop(pricesUS.loc[:, 'c2':'c39'], axis=1)
columns =  pricesUS.loc[:, 'c2':'c39'].loc[0:22]
new_col = columns/columns.loc[14]
pricesUS_2 = pd.concat([pricesUS_1, new_col], axis=1).rename(columns={"c1": "year"})
#pricesUS_2

pricesUS_2.rename(columns={'c2':'c1', 'c3':'c2', 'c4':'c3', 'c5':'c4', 'c6':'c5', 'c7':'c6', 'c8':'c7', 'c9':'c8',
                             'c10':'c9', 'c11':'c10', 'c12':'c11', 'c13':'c12','c14':'c13', 'c15':'c14', 'c16':'c15',
                             'c17':'c16', 'c18':'c17', 'c19':'c18', 'c20':'c19', 'c21':'c20','c22':'c21', 'c23':'c22',
                             'c24':'c23', 'c25':'c24', 'c26':'c25', 'c27':'c26', 'c28':'c27', 'c29':'c28', 'c30':'c29',
                             'c31':'c30','c32':'c31', 'c33':'c32', 'c34':'c33', 'c35':'c34', 'c36':'c35', 'c37':'c36',
                             'c38':'c37', 'c39':'c38'}, inplace=True)

In [63]:
temp2 = pd.read_stata('temp26')
pricesUS_3 = pd.merge(pricesUS_2,temp2, on='year')
#pricesUS_3

In [64]:
pricesUS_3['id'] = np.arange(1, len(pricesUS_3)+1)
pricesUS_3_wl = pd.wide_to_long(pricesUS_3,["c", "share"], i='id', j="ind")
pricesUS_3_wl = pricesUS_3_wl.sort_values(by=['id','ind']).drop(columns=['index'])
#pricesUS_3_wl

In [65]:
skillintUS = pd.read_stata('skillintUS6.dta', convert_categoricals=False)
pricesUS_4 = pd.merge(pricesUS_3_wl,skillintUS, on='ind')
#pricesUS_4

In [66]:
#some industries match to multiple goods
pricesUS_4.loc[(pricesUS_4['ind'] == 7), 'ratio80'] = 0.2395
pricesUS_4.loc[(pricesUS_4['ind'] == 7) | (pricesUS_4['ind'] ==28), 'ratio80'] = 0.539
pricesUS_4.loc[(pricesUS_4['ind'] == 5), 'ratio80'] = 0.265

In [67]:
#weighted average of prices of skill-intensive/low-skill-intensive items
pricesUS_4['HH'] = 0
pricesUS_4.loc[(pricesUS_4['ratio80'] >0.37), 'HH'] = 1
#pricesUS_4

In [68]:
pricesUS_5 = pricesUS_4.groupby(['year','HH'])['c'].mean()
pricesUS_55 = pd.DataFrame(pricesUS_5).reset_index('HH').reset_index('year')
pricesUS_55['id'] = [1,2,3,4,5,6,7,8]
#pricesUS_55

In [69]:
pricesUS_6 = pd.read_stata('ltw6.dta').fillna(0)
pricesUS_6 = pricesUS_6.groupby('year')['c0', 'c1'].sum().reset_index()
pricesUS_6['lplow'] =np.log(pricesUS_6['c0'])
pricesUS_6['lphigh'] = np.log(pricesUS_6['c1']) - pricesUS_6['lplow']
pricesUS_6['year'] = pricesUS_6['year'] - 1900 
#pricesUS_6
pricesUS_6.to_stata('prices6')

<ipython-input-69-db5ab58caeb6>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pricesUS_6 = pricesUS_6.groupby('year')['c0', 'c1'].sum().reset_index()


In [70]:
sampleUS = pd.read_stata('sampleUS')
sampleUS_1 = pd.merge(sampleUS,pricesUS_6, on='year')
#sampleUS_1

In [71]:
sampleUS_1.rename(columns={'foodhome':'share1', 'foodout':'share2', 'tobacco':'share3', 'alcohol':'share4', 
                             'niteclub':'share5', 'clothes':'share6','tailors':'share7', 'jewelry':'share8', 
                             'hlthbeau':'share9', 'renthome':'share10', 'furnish':'share11', 'elect':'share12', 
                             'gas':'share13','water':'share14', 'homefuel':'share15','telephon':'share16', 
                             'servants':'share17', 'drugs':'share18', 'orthopd':'share19', 'doctors':'share20', 
                             'hospital':'share21', 'helthins':'share22', 'busiserv':'share23', 'lifeins':'share24',
                             'parts':'share25','carservs':'share26', 'gasoline':'share27', 'autoins':'share28', 
                             'masstran':'share29', 'othtrans':'share30', 'airfare':'share31', 'pubs':'share32', 
                             'recsport':'share33', 'othrec':'share34', 'highedu':'share35','lowedu':'share36', 
                             'othedu':'share37', 'charity':'share38'}, inplace=True)

In [72]:
sampleUS_1['lpallit'] = np.log(sampleUS_1['cpi_00'])

In [73]:
sampleUS_1 = sampleUS_1[['year', 'share1', 'share2', 'share3', 'share4', 'share5', 'share6', 'share7', 'share8', 'share9',
                               'share10', 'share11', 'share12', 'share13', 'share14', 'share15', 'share16', 'share17', 
                               'share18', 'share19', 'share20', 'share21', 'share22', 'share23', 'share24', 'share25',
                               'share26', 'share27', 'share28', 'share29', 'share30', 'share31', 'share32', 'share33', 
                               'share34', 'share35', 'share36', 'share37', 'share38', 'ltotexp', 'exptotal', 'age', 'ee',
                             'lpallit', 'lphigh', 'sex', 'nchild']]
sampleUS_1['id'] = np.arange(1, len(sampleUS_1)+1)
#sampleUS_1

In [74]:
#wide to long 
sampleUS_2 = pd.wide_to_long(sampleUS_1,["share"], i='id', j="inden")
#sampleUS_2

In [75]:
sampleUS_2 = sampleUS_2.reset_index('inden').sort_values(by=['id','inden'])
sampleUS_2['ind'] = sampleUS_2['inden']
#sampleUS_2

In [76]:
sampleUS_3 = pd.merge(sampleUS_2,skillintUS, on='ind')
#sampleUS_3

In [77]:
sampleUS_3.loc[(sampleUS_3['sex'] == 1), 'sex1'] = 0
sampleUS_3.loc[(sampleUS_3['sex'] == 2), 'sex1'] = 1

In [78]:
sampleUS_3['HH'] = 0
sampleUS_3.loc[(sampleUS_3['ratio80'] >0.37), 'HH'] = 1

sampleUS_3['LL'] = 0
sampleUS_3.loc[(sampleUS_3['ratio80'] <= 0.37), 'LL'] = 1
#sampleUS_3

In [79]:
sampleUS_3.loc[(sampleUS_3['ee'] == 1) & (sampleUS_3['ratio80'] > 0.37), 'exphh'] = 1
sampleUS_3.loc[sampleUS_3.exphh == 1, 'exphh'] = 6.75e+09

sampleUS_3.loc[sampleUS_3['ratio80'] > 0.37, 'exphigh'] = 1
sampleUS_3.loc[sampleUS_3.exphigh == 1, 'exphigh'] = 1.11e+10
#sampleUS_3

In [80]:
#THIS GIVES R1 FOR CALIBRATION
sampleUS_3['R1'] = sampleUS_3['exphh']/sampleUS_3['exphigh'] 
sampleUS_3[['R1']].describe()

,R1
count,2.066180e+05
mean,6.081081e-01
std,1.347148e-12
min,6.081081e-01
25%,6.081081e-01
50%,6.081081e-01
75%,6.081081e-01
max,6.081081e-01


In [81]:
sampleUS_3['ltotexpLL']= sampleUS_3['ltotexp']*sampleUS_3['LL'] 
sampleUS_3['lphighLL']=sampleUS_3['lphigh']*sampleUS_3['LL'] 
sampleUS_3['ltotexpHH']=sampleUS_3['ltotexp']*sampleUS_3['HH'] 
sampleUS_3['lphighHH']=sampleUS_3['lphigh']*sampleUS_3['HH']

sampleUS_3['ageLL']= sampleUS_3['age']*sampleUS_3['LL']
sampleUS_3['sexLL']= sampleUS_3['sex']*sampleUS_3['LL']
sampleUS_3['nchildLL']= sampleUS_3['nchild']*sampleUS_3['LL']

sampleUS_3['ageHH']= sampleUS_3['age']*sampleUS_3['HH']
sampleUS_3['sexHH']= sampleUS_3['sex']*sampleUS_3['HH']
sampleUS_3['nchildHH']= sampleUS_3['nchild']*sampleUS_3['HH']

In [82]:
#sampleUS_3

In [83]:
sampleUS_4 = pd.read_stata('t6reg1.dta')
sampleUS_4['incL'] = sampleUS_4['inc_elast_L'].max()
sampleUS_4['incH'] = sampleUS_4['inc_elast_H'].max()

sampleUS_4['priceL'] = sampleUS_4['price_elast_L'].max()
sampleUS_4['priceH'] = sampleUS_4['price_elast_H'].max()

sampleUS_4 = sampleUS_4.drop(columns=['inc_elast_L', 'price_elast_L', 'inc_elast_H', 'price_elast_H'])
sampleUS_4.rename(columns={'incL':'inc_elast_L','incH':'inc_elast_H','priceL':'price_elast_L','priceH':'price_elast_H'}, inplace=True)
#sampleUS_4

In [84]:
sampleUS_4[sampleUS_4['ee'] == 0]['mshare'].describe()

count    22288.0
mean         0.5
std          0.0
min          0.5
25%          0.5
50%          0.5
75%          0.5
max          0.5
Name: mshare, dtype: float64

In [85]:
sampleUS_4[sampleUS_4['ee'] == 1]['mshare'].describe()

count    24308.0
mean         0.5
std          0.0
min          0.5
25%          0.5
50%          0.5
75%          0.5
max          0.5
Name: mshare, dtype: float64

In [86]:
#from Table 5
sampleUS_4['lH'] = 1.96
sampleUS_4['lL'] = 0.45
sampleUS_4['a1'] = 0.80
sampleUS_4['a2'] = 0.43

sampleUS_4['ratio80'] = 1.22 #/*H/L*/
sampleUS_4['ratio'] = 0.55 #/*H/H+L*/
#sampleUS_4

In [87]:
def T(R1,a1,a2,price_elast_H,inc_elast_H,price_elast_L,inc_elast_L):
    T = R1*((a1-a2)*price_elast_H+(1-a2)*inc_elast_H)+(1-R1)*((a1-a2)*price_elast_L-a2*inc_elast_L)
    return T

sampleUS_4['T'] = T(sampleUS_4['R1'], sampleUS_4['a1'],sampleUS_4['a2'],
                    sampleUS_4['price_elast_H'],sampleUS_4['inc_elast_H'],
                    sampleUS_4['price_elast_L'],sampleUS_4['inc_elast_L'])

In [88]:
def result(a1,a2,lH,lL,R1,ratio80,T):
    result=(1-a2)*((lH-lL)*(R1-(1-R1)*ratio80)-(1+lH+ratio80*(1+lL)))/((lL+1)*1.4+(lH-lL)*(1-a1)*1.4+(lH-lL)*T)
    return result

sampleUS_4['result'] = result(sampleUS_4['a1'],sampleUS_4['a2'],sampleUS_4['lH'],sampleUS_4['lL'],  
                    sampleUS_4['R1'],sampleUS_4['ratio80'],sampleUS_4['T']) 

In [89]:
sampleUS_4[['result']].describe()

,result
count,1.215400e+04
mean,-1.121195e+00
std,1.381174e-13
min,-1.121195e+00
25%,-1.121195e+00
50%,-1.121195e+00
75%,-1.121195e+00
max,-1.121195e+00


In [90]:
#counterfactual
def counter(a1,a2,lH,lL,ratio,ratio80,price_elast):
    counter=-(1-a2)*(1+lH+ratio80*(1+lL))/((lL+1)*1.4+(lH-lL)*(1-a1)*1.4+(lH-lL)*(price_elast*(a1-a2)+ratio-a2))
    return counter

sampleUS_4['counter'] = counter(sampleUS_4['a1'],sampleUS_4['a2'],sampleUS_4['lH'],sampleUS_4['lL'],  
                    sampleUS_4['ratio'],sampleUS_4['ratio80'],sampleUS_4['price_elast']) 

In [91]:
#sampleUS_4[['counter']].describe()

In [92]:
#DECOMPOSITION
#The direct effect of education elasticities can be quantified in 

def edd(a2,lH,lL,R1,ratio80):
    edd=(1-a2)*(lH-lL)*(R1-(1-R1)*ratio80)
    return edd

sampleUS_4['edd'] = edd(sampleUS_4['a2'],sampleUS_4['lH'],sampleUS_4['lL'],sampleUS_4['R1'],sampleUS_4['ratio80'])
#sampleUS_4[['edd']].describe()

In [93]:
#The difference in income elasticities is calculated at
def incc(R1,a2,inc_elast_H,inc_elast_L):
    incc= (R1*(1-a2)*inc_elast_H-(1-R1)*a2*inc_elast_L)
    return incc

sampleUS_4['incc'] = incc(sampleUS_4['R1'],sampleUS_4['a2'],sampleUS_4['inc_elast_H'],sampleUS_4['inc_elast_L']) 
#sampleUS_4[['incc']].describe()

In [94]:
#The difference in price elasticities is calculated 
def priced(R1,a1,a2,price_elast_H,price_elast_L, price_elast):
    priced= (R1*(a1-a2)*price_elast_H+(1-R1)*(a1-a2)*price_elast_L-(a1-a2)*price_elast)
    return priced

sampleUS_4['priced'] = priced(sampleUS_4['R1'],sampleUS_4['a1'],sampleUS_4['a2'],sampleUS_4['price_elast_H'],
                              sampleUS_4['price_elast_L'],sampleUS_4['price_elast']) 
#sampleUS_4[['priced']].describe()

In [95]:
sampleUS_4[['result','counter','edd','incc','priced']].describe()

,result,counter,edd,incc,priced
count,1.215400e+04,4.659600e+04,1.215400e+04,1.215400e+04,1.215400e+04
mean,-1.121195e+00,-1.217543e+00,1.125083e-01,1.956781e-01,-1.614050e-02
std,1.381174e-13,1.121337e-12,2.012362e-14,1.904111e-14,4.559041e-15
min,-1.121195e+00,-1.217543e+00,1.125083e-01,1.956781e-01,-1.614050e-02
25%,-1.121195e+00,-1.217543e+00,1.125083e-01,1.956781e-01,-1.614050e-02
50%,-1.121195e+00,-1.217543e+00,1.125083e-01,1.956781e-01,-1.614050e-02
75%,-1.121195e+00,-1.217543e+00,1.125083e-01,1.956781e-01,-1.614050e-02
max,-1.121195e+00,-1.217543e+00,1.125083e-01,1.956781e-01,-1.614050e-02


In [96]:
#persentage terms
pt1 = sampleUS_4['result'].mean()*0.66
pt2 = sampleUS_4['counter'].mean()*0.66
pt3 = pt1 - pt2
pt4 = 0.13- pt2
pt5 = pt3/pt4

In [97]:
#TABLE6
USCEX = [-1.12,-1.22,'','','']
Percentage_terms = [pt1,pt2,pt3,pt4,pt5]

index = ['US CEX','Percentage terms']

rows = [USCEX,Percentage_terms]
#columns=['Model with income and education elasticities','Model without income and education elasticities','Difference','Demand shift','Contribution of elasticities']

TABLE6 = pd.DataFrame(data = rows, index=index, columns=['one','two','three','four','five'])                                                          
TABLE6.round(2)

,one,two,three,four,five
US CEX,-1.12,-1.22,,,
Percentage terms,-0.74,-0.80,0.0635897,0.933578,0.0681139
